In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
a = ["a","b","c"]
#b=b.reshape((1,-1))
c = [["b"],["c"]]

In [7]:
class OnehotEncoder:
    def __init__(self):
        self.lenght=0
        self.data={}
    def fit(self,data):
        for id,key in enumerate(data):
            self.data[key]=id
        self.lenght = len(data)
    def transform(self,data):
        ba = []
        for i in data:
            a = np.zeros(self.lenght,dtype=np.int32)
            for j in i:
                a[self.data[j]]=1
            ba.append(a.tolist())
        return ba

In [8]:
b = OnehotEncoder()

In [9]:
b.fit(a)

In [10]:
b.transform(c)

[[0, 1, 0], [0, 0, 1]]

In [95]:
class Model_AI:
    def __init__(self):
        self.data=None
        self.encoder={}
        self.best_model=None
        self.features = []
        self.str_col=[]
    def check_str_type(self, word):
        return str(type(word)).split()[1].split("'")[1] == "str"
    def process_data(self, data_path):
        self.data = pd.read_csv(data_path)
        self.origin_data = self.data.copy()
        self.features.append(list(self.data.columns))
        self.target = self.data.columns[-1]
        for i in self.data.columns:
            if self.check_str_type(self.data.iloc[0][i]):
                LaEn = OnehotEncoder()
                temp = self.data[i].unique()
                LaEn.fit(temp)
                self.str_col.append(i)
                temp_data = np.array(LaEn.transform(self.data[i].values.reshape((-1,1)))).T
                for id, key in enumerate(temp):
                    self.data[key]=temp_data[id]
                self.encoder.update({i:LaEn})
                self.data.drop(i,axis=1,inplace=True)
        return (self.target,list(filter(lambda x: x!=self.target, self.data.columns)))
    def fit(self, data_path):
        rank={}
        target, feature = self.process_data(data_path)
        X = self.data[feature].values
        y = self.data[[target]].values
        xtrain,xtest,ytrain,ytest = train_test_split(X,y, test_size = 0.3)
        Linear_model = LinearRegression()
        Linear_model.fit(xtrain,ytrain)
        rank.update({Linear_model.score(xtest,ytest):Linear_model})
        
        svr_model = SVR(kernel="poly", degree=5)
        svr_model.fit(xtrain,ytrain)
        rank.update({svr_model.score(xtest,ytest):svr_model})
        
        rfr_model = RandomForestRegressor(n_estimators=500)
        rfr_model.fit(xtrain,ytrain)
        rank.update({rfr_model.score(xtest,ytest):rfr_model})
        self.best_model = rank[max(list(rank.keys()))]
    def extract_vector(self,features):
        feature_vector = []
        for i in features.keys():
            if i not in self.str_col:
                feature_vector.append(features[i])
        for i in self.str_col:
            feature_vector.extend(self.encoder[i].transform([[features[i]]])[0])
        return feature_vector
    def predict(self, features):
        '''
            features = { Position:..., Level:...}
        '''
        features = self.extract_vector(features)
        #return features
        return self.best_model.predict([features])


In [99]:
%cd "C:\Users\Dang\Desktop\Python4ML_UIT"

C:\Users\Dang\Desktop\Python4ML_UIT


In [100]:
a = Model_AI()

In [101]:
path = "./Lab_1/data/Position_Salaries.csv"

In [102]:
a.fit(path)

C:\Users\Dang\anaconda3\envs\CS116\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Dang\AppData\Local\Temp\ipykernel_11000\3856323768.py:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr_model.fit(xtrain,ytrain)


In [103]:
a.best_model

RandomForestRegressor(n_estimators=500)

In [108]:
feature = {"Position":"Partner","Level":2}

In [109]:
print(a.predict(feature)[0])

88160.0


In [110]:
a=np.array([[0]])
a

array([[0]])

In [111]:
a.reshape(1)

array([0])